1. Remove irrelevant columns
1. Impute missing values
2. One-Hot Encode Strings
3. Log transform non-normal variables
4. Remove irrelevant columns
5. (Try) Scaling


In [11]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [2]:
# Load data
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HousePrices2/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HousePrices2/test.csv')

In [ ]:
plt.figure(figsize=(25,25))
sb.heatmap(train.corr(), cmap='coolwarm', annot=True, linewidths=0.5)
plt.show()

In [3]:
# Drop non correlant features with SalePrice
train = train.drop(['Id', 'MSSubClass', 'BsmtFinSF2', 'LowQualFinSF', 'BsmtHalfBath', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'], axis="columns")
test = test.drop(['Id', 'MSSubClass', 'BsmtFinSF2', 'LowQualFinSF', 'BsmtHalfBath', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'], axis="columns")

In [4]:
# Drop missing values
train = train.drop(['PoolQC', 'MiscFeature', 'Alley'], axis="columns")
test = test.drop(['PoolQC', 'MiscFeature', 'Alley'], axis='columns')

In [5]:
# Impute missing numerical values
def find_columns_with_missing_values(df):
    missing_columns = df.columns[df.isna().any()].tolist()
    return missing_columns

def handle_missing_value(df, column):
    if pd.api.types.is_numeric_dtype(df[column]):
        df[column].fillna(df[column].mean(), inplace=True)
    return df

missing_columns = find_columns_with_missing_values(train)

for col in missing_columns:
      df = handle_missing_value(train, col)
      train = df

In [6]:
# Impute categorical variables with mean category (mode)
missing_categorical_values = find_columns_with_missing_values(train)

for val in missing_categorical_values:
  train[val] = train[val].fillna(train[val].mode()[0])

In [7]:
# One-Hot encode categorical variables
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [ ]:
train.hist(figsize=(20,15))

In [8]:
# Log transform positive numerical variables
cols = ['LotArea', 'OverallCond', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', '2ndFlrSF', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch']

for col in cols:
  mask = train[col] > 0
  train.loc[mask, col] = np.log(train.loc[mask, col])

In [ ]:
train.hist(figsize=(20,15))

In [9]:
# Train-Test split
X = train.drop(['SalePrice'], axis="columns")
y = train['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Train and Score
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.914369904651672